In [111]:
import pandas as pd
import numpy as np
from datetime import date

In [107]:
DATA_DIR = '../epimodel-covid-data/'

In [81]:
cm = pd.read_csv('COVID 19 Containment measures data.csv',parse_dates=['Date Start','Date end intended']).dropna(subset=['Country'])
DATE = '2020_04_23'
complete = pd.read_csv('Dataset completeness.csv')

complete = complete[~pd.isna(complete['Complete up to date'])]['Country'].append(pd.Series(['United States']))

masks = pd.read_csv('mask_survey_data.csv').drop(columns=['Unnamed: 0']).rename(columns={'country':'Country',
                                                                                        'value':'Quantity',
                                                                                        'date':'Date Start'})
complete = complete.append(masks[masks['Country'].str.startswith('United States')]['Country'])
cm = cm[cm['Country'].isin(complete)]
masks = masks[masks['Country'].isin(complete)]
ms = masks.groupby(['Country','Date Start']).mean().reset_index()
ms['Keywords'] = 'public mask wearing data'
ms['Source'] = 'Survey data'
ms['Date Start'] = pd.to_datetime(ms['Date Start'])

cm.loc[cm['Country'].str.startswith('United States')
       &(~pd.isna(cm['Implementing State/Province'])),['Country']] = cm.loc[cm['Country'].str.startswith('United States')
                                                                            &(~pd.isna(cm['Implementing State/Province'])),['Country']] +' - ' +cm.loc[cm['Country'].str.startswith('United States')
                                                                                                            &(~pd.isna(cm['Implementing State/Province'])),['Implementing State/Province']].values

In [105]:
acaps = pd.read_csv('/home/guest/Downloads/ef_acaps_combined.csv')

# Bodgy mask features

acaps.loc[1116,'Quantity'] = 5
acaps.loc[1731,'Quantity'] = 40
acaps.loc[(acaps['Keywords']=='public mask wearing data')&(pd.isna(acaps['Quantity'])),'Quantity']=80


acaps['Date Start'] = pd.to_datetime(acaps['Date Start'])
acaps = acaps.dropna(subset=['Date Start'])
acaps_mask_data = acaps[(acaps['Keywords']=='public mask wearing data')&(acaps['DataSource']=='ACAPS')]
acaps_mask_data = acaps_mask_data[acaps_mask_data['Country'].isin(complete)]

In [106]:
# If you want all acaps data

cm = pd.concat([cm,ms,acaps])

# Or just the mask data

# cm = pd.concat([cm,ms,acaps_mask_data])
cm = cm.dropna(subset=['Date Start'])
cm['Date Start'] = pd.to_datetime(cm['Date Start'])
cm['Date Start'].unique()

array(['2020-03-16T00:00:00.000000000', '2020-02-01T00:00:00.000000000',
       '2020-03-20T00:00:00.000000000', '2020-03-14T00:00:00.000000000',
       '2020-02-26T00:00:00.000000000', '2020-03-18T00:00:00.000000000',
       '2020-03-21T00:00:00.000000000', '2020-03-17T00:00:00.000000000',
       '2020-03-13T00:00:00.000000000', '2020-03-11T00:00:00.000000000',
       '2020-03-10T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
       '2020-03-05T00:00:00.000000000', '2020-03-04T00:00:00.000000000',
       '2020-03-02T00:00:00.000000000', '2020-03-01T00:00:00.000000000',
       '2020-02-29T00:00:00.000000000', '2020-02-27T00:00:00.000000000',
       '2020-02-25T00:00:00.000000000', '2020-02-24T00:00:00.000000000',
       '2020-02-23T00:00:00.000000000', '2020-02-22T00:00:00.000000000',
       '2020-02-21T00:00:00.000000000', '2020-02-19T00:00:00.000000000',
       '2020-02-18T00:00:00.000000000', '2020-02-07T00:00:00.000000000',
       '2020-02-04T00:00:00.000000000', '2020-01-31

In [101]:
jhcc = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jhd = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
JH2CM = {
    'Korea, South':'South Korea',
    'Taiwan*':'Taiwan',
    ('China','Hong Kong'):'Hong Kong',
    ('China','Macau'):'Macau',
    ('Denmark','Faroe Islands'):'Faroe Islands',
    'US':'United States',
    'North Macedonia':'Macedonia'
}


def jh2cm(c,s):
    if c in JH2CM:
        return JH2CM[c]
    elif (c,s) in JH2CM:
        return JH2CM[(c,s)]
    return c

def pre_jh(jh,val="Confirmed Cases"):
    jh['Country/Region'] = [jh2cm(c,s) for c,s in jh[['Country/Region','Province/State']].values]
    jh['Country/Region'] = [jh2cm(c,s) for c,s in jh[['Country/Region','Province/State']].values]

    jh.drop(columns=['Lat','Long'],inplace=True)
    jh = jh.groupby('Country/Region').sum().reset_index()
    jh = pd.melt(jh,id_vars=['Country/Region'],value_vars=jh.columns[2:],value_name=val,var_name='Date')
    jh['Date'] = pd.to_datetime(jh['Date'])
    return jh

jhcc, jhd = pre_jh(jhcc),pre_jh(jhd,val='Deaths')
jhcc['Deaths'] = jhd['Deaths']

In [93]:
MAX_COLS = {
    'Symptomatic isolation - targeted':{'contact isolation - symptoms':1,
                                        'cohort isolation - symptoms':1},
    'Symptomatic isolation - blanket':{'cluster isolation - symptoms':1,
                                       'blanket isolation - symptoms':2},
    'Asymptomatic isolation - targeted':{'contact isolation - no symptoms':1,
                                         'cohort isolation - no symptoms':2},
    'Asymptomatic isolation - blanket':{'cluster isolation - no symptoms':1,
                                        'blanket isolation - no symptoms':3,
                                        'blanket curfew - no symptoms':2,
                                        'natural village quarantine':3},
    'Domestic travel restriction':{'domestic traveller quarantine':1,
                                   'domestic travel ban':2,
                                   'total vehicle ban':2},
    'Nonessential business suspension':{'general nonessential business suspension':1,
                                        'limited nonessential business suspension':0.5,
                                   'remote work':0.5},
    'International travel restriction':{'international traveller screening - risk countries':1,
                                        'international traveller screening - all countries':2,
                                        'international traveller quarantine - risk countries':3,
                                        'international traveller quarantine - all countries':4,
                                        'international travel ban - risk countries':5,
                                        'international travel ban - all countries':6},
    'Testing':{'testing numbers total':np.nan},
    'Contact tracing':{'contacts traced total':np.nan},
    'Mask wearing':{'public mask wearing data':np.nan},
    'Hand washing':{'public handwashing data':np.nan}
    
}

MIN_COLS = {
    'Gatherings banned':['indoor gatherings banned',
                        'outdoor gatherings banned']
}

CUMSUM_COLS = {
    'Healthcare specialisation':['clinic specialisation',
                                'case transport',
                                'quarantine zone',
                                'hospital specialisation',
                                'healthcare entry screening',
                                'remote medical treatment',
                                'visiting in hospital banned'],
    'Phone hotline':['phone line'],
    'Assisting people to stay home':['unemployment benefits extension',
                                    'eviction moratorium',
                                    'isolation allowance',
                                    'compulsory isolation'],
    'Public cleaning':['public transport cleaning',
                      'public facility cleaning'],
    'Miscellaneous hygiene measures':['funeral hygiene',
                                     'cash cleaning',
                                     'cash banned'],
    'Social distancing and hygiene advice':['risk communication',
                                         'community engagement',
                                         'coronavirus education activities',
                                         'handshakes banned',
                                         'social distancing advice',
                                         'stay home advice',
                                         'space minimum',
                                         'outdoor person density',
                                         'indoor person density',
                                         'public venue screening',
                                         'handwashing encouragement',
                                         'public mask encouragement'],
    'Hygiene supply':['public mask supply',
                     'public mask and hygiene supply',
                     'public hand sanitizer supply',
                     'public venue screening'],
    'School closure':['school closure',
                     'university closure',
                     'nursery school closure',
                     'remote schooling',
                     'public transport stopped'],
    'Activity cancellation':['activity cancellation - other',
                            'sports cancellation',
                            'religious activity cancellation',
                            'religious activity limitations',
                            'weddings canceled',
                            'very large event cancellation or postponement',
                            'cultural activity limitation',
                            'remote cultural content',
                            'restaurant limitations',
                            'closure of gathering places'],
    'Resumption':['public transport resumed',
                 'activity resumed',
                 'business resumed'],
    'Diagnostic criteria loosened':['diagnostic criteria loosened'],
    'Diagnostic criteria tightened':['diagnostic criteria tightened']    
}

TEST_COLS = {    
    'Testing criteria':{'test all':1,
                       'test symptomatic':0.5,
                       'cluster testing':0.3,
                       'test contacts':0.1,
                       'test cohorts':0.2,
                       'test travellers':0.1,
                       'test medical staff':0.1,
                       'test vulnerable':0.1}
}

def default_values(kw):
    for k, v in {**MAX_COLS,**TEST_COLS}.items():
        if (kw in v) and (v[kw]!=np.nan):
            return v[kw]
    return np.nan

def keywords(kws_quants):
    res =  pd.DataFrame([(i,j[1]) 
                         for j in kws_quants.values 
                         for i in str(j[0]).split(', ')],
                        columns=['Keywords','Quantity'])
    res['Quantity'] = res['Keywords'].apply(default_values).fillna(res['Quantity'])
    return res

def sum_kws(kws_quants,tags):
    return pd.Series(kws_quants['Keywords'].unique()).isin(tags).sum()

def max_kws(kws_quants,tags):
    return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].max()

def min_kws(kws_quants,tags):
    return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].min()

def test_kws(kws_quants,tags):
    if 'test all' in kws_quants['Keywords']:
        return 1
    elif 'test symptomatic' in kws_quants['Keywords']:
        return 0.5
    else:
        return kws_quants[kws_quants['Keywords'].isin(tags)]['Quantity'].sum()


jdict = {**MAX_COLS,**MIN_COLS,**CUMSUM_COLS,**TEST_COLS}


In [94]:
data_dict = {k:[] for k in jdict.keys()}
data_dict['Date'] = []
data_dict['Country'] = []

for d in pd.date_range(cm['Date Start'].min(),pd.Timestamp('2020-04-23')):
    print(d,' of ',cm['Date Start'].max())
    for c in cm['Country'].unique():
        data_dict['Country'].append(c)
        data_dict['Date'].append(d)
        if ((cm['Date Start']<=d)&(cm['Country']==c)).any():
            kws_quants = keywords(cm[(cm['Date Start']<=d)&(cm['Country']==c)][['Keywords','Quantity']])
        else:
            kws_quants = pd.DataFrame({'Keywords':[],'Quantity':[]})
        for col in MAX_COLS:
            data_dict[col].append(max_kws(kws_quants,MAX_COLS[col].keys()))
        for col in CUMSUM_COLS:
            data_dict[col].append(sum_kws(kws_quants,CUMSUM_COLS[col]))
        for col in TEST_COLS:
            data_dict[col].append(test_kws(kws_quants,TEST_COLS[col].keys()))
        for col in MIN_COLS:
            data_dict[col].append(min_kws(kws_quants,MIN_COLS[col]))
            

2019-12-18 00:00:00  of  2020-12-04 00:00:00
2019-12-19 00:00:00  of  2020-12-04 00:00:00
2019-12-20 00:00:00  of  2020-12-04 00:00:00
2019-12-21 00:00:00  of  2020-12-04 00:00:00
2019-12-22 00:00:00  of  2020-12-04 00:00:00
2019-12-23 00:00:00  of  2020-12-04 00:00:00
2019-12-24 00:00:00  of  2020-12-04 00:00:00
2019-12-25 00:00:00  of  2020-12-04 00:00:00
2019-12-26 00:00:00  of  2020-12-04 00:00:00
2019-12-27 00:00:00  of  2020-12-04 00:00:00
2019-12-28 00:00:00  of  2020-12-04 00:00:00
2019-12-29 00:00:00  of  2020-12-04 00:00:00
2019-12-30 00:00:00  of  2020-12-04 00:00:00
2019-12-31 00:00:00  of  2020-12-04 00:00:00
2020-01-01 00:00:00  of  2020-12-04 00:00:00
2020-01-02 00:00:00  of  2020-12-04 00:00:00
2020-01-03 00:00:00  of  2020-12-04 00:00:00
2020-01-04 00:00:00  of  2020-12-04 00:00:00
2020-01-05 00:00:00  of  2020-12-04 00:00:00
2020-01-06 00:00:00  of  2020-12-04 00:00:00
2020-01-07 00:00:00  of  2020-12-04 00:00:00
2020-01-08 00:00:00  of  2020-12-04 00:00:00
2020-01-09

In [102]:
# Write 27 features data to file

unmerged_data = pd.DataFrame(data_dict).dropna(subset=['Country'])

djh = unmerged_data[unmerged_data['Country'].isin(jhcc['Country/Region'])]

jhcc.rename(columns={'Country/Region':'Country'},inplace=True)
jh_merged_data = djh.merge(jhcc,on=['Date','Country'])

unmerged_data.to_csv(f'countermeasures_features_{DATE}-acaps.csv')
jh_merged_data.to_csv(f'countermeasures_db_johnshopkins_{DATE}-acaps.csv')

In [112]:
# Select features for modelling

REPORTCOLS_BOOL = {'Gatherings limited to 10':('Gatherings banned',lambda x:(x<=10)&(x>0)),
                   'Gatherings limited to 100':('Gatherings banned',lambda x:(x<=100)&(x>0)),
                   'Gatherings limited to 1000':('Gatherings banned',lambda x: (x<=1000)&(x>0)),
                  'Business suspended - some':('Nonessential business suspension',lambda x: x>=0.5),
                  'Business suspended - many':('Nonessential business suspension',lambda x: x>=1),
                  'Schools and universities closed':('School closure',lambda x: x>=1),
                  'General curfew':('Asymptomatic isolation - blanket',lambda x: x>=2),
                  'Healthcare specialisation':('Healthcare specialisation',lambda x: x>=2),
                  'Mask wearing over 70%':('Mask wearing',lambda x: x>=70)}
data0 = unmerged_data.copy()


for rc, (vc, f) in REPORTCOLS_BOOL.items():
    data0[rc] = data0[vc].apply(f).astype(float)

model_data = data0[list(REPORTCOLS_BOOL.keys())+['Country','Date']]
model_data = model_data[pd.to_datetime(model_data['Date'])<=pd.Timestamp(date.today())]

regions = pd.read_csv(f'{DATA_DIR}/regions.csv')

model_data = model_data[model_data['Country'].isin(regions['Name'])]
model_data.loc[:,'Country'] = model_data['Country'].apply(lambda x: regions[regions['Name']==x]['Code'].iloc[0])

# For some reason Namibia gets NaN as a code

model_data['Country'] = model_data['Country'].replace(np.nan,'NA')
model_data = model_data.rename(columns={'Country':'Code'})
model_data[~model_data['Country'].str.startswith('United States')].to_csv(f'countermeasures-model-boolean-{DATE}.csv')